In [ ]:
import os
import cv2
import numpy as np


IN_BED_DIR = '/home/jeff/Dropbox/data/persistent_alarm/in_bed'
OUT_BED_DIR = '/home/jeff/Dropbox/data/persistent_alarm/not_in_bed'
X = None
y = None
for file in os.listdir(IN_BED_DIR):
    file_path = os.path.join(IN_BED_DIR, file)
    if not os.path.isfile(file_path):
        continue
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = img.reshape(1, img.shape[0], img.shape[1], 1)
    if X is None:
        X = img
        y = np.array([1])
    else:
        X = np.append(X, img, axis=0)
        y = np.append(y, [1], axis=0)
for file in os.listdir(OUT_BED_DIR):
    file_path = os.path.join(OUT_BED_DIR, file)
    if not os.path.isfile(file_path):
        continue
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = img.reshape(1, img.shape[0], img.shape[1], 1)
    X = np.append(X, img, axis=0)
    y = np.append(y, [0], axis=0)

print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
print(y_train[:2])

In [ ]:
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD

# input
model_input = Input(shape=(128,171,1))

# hidden layers
# Block 1
x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block1_conv1')(model_input)
x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

x = Flatten(name='flatten')(x)
x = Dense(100, activation='relu', name='fc1')(x)
x = Dense(100, activation='relu', name='fc2')(x)
x = Dense(1, name='pre-predictions')(x)
x = Activation('sigmoid')(x)

sgd = SGD(lr=0.01)
model = Model(model_input, x)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_split=0)

In [ ]:
import keras.backend as K
import tensorflow as tf

# calculate gradients
def get_gradients(model):
    """Return the gradient of every trainable weight in model

    Parameters
    -----------
    model : a keras model instance

    First, find all tensors which are trainable in the model. Surprisingly,
    `model.trainable_weights` will return tensors for which
    trainable=False has been set on their layer (last time I checked), hence the extra check.
    Next, get the gradients of the loss with respect to the weights.

    """
    weights = [tensor for tensor in model.trainable_weights]
    optimizer = model.optimizer

    return optimizer.get_gradients(model.total_loss, weights)


gradients = get_gradients(model)
sess = tf.Session()
sess.run(tf.initialize_all_variables())
evaluated_gradients = sess.run()

In [ ]:
print(model.predict(X_train[:10]))

model2 = Model(inputs=model.input, outputs=model.get_layer('fc2').output)
model2.predict(X_train[:10])